In [17]:
import sqlite3
from sqlite3 import Error
from sqlalchemy import create_engine
import pandas as pd
import pymysql.cursors

In [49]:
db_file = 'sqlite_db_pythonsqlite.db'

In [46]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

def create_connection2(db_file):
    engine = create_engine(db_file)
    conn = engine.connect()
    return conn

def get_tables(db_file):
    engine = create_engine(db_file)
    table_names = engine.table_names()
    print(table_names)

def get_data(db_file, sql):
    conn = sqlite3.connect(db_file)
    cur = conn.cursor()
    cur.execute(sql)
    data = cur.fetchall()
    headers = [i[0] for i in cur.description]
    print(headers)
    for row in data:
        print(row)
 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT *
        FROM FACILITIES
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
(0, 'Tennis Court 1', 5, 25, 10000, 200)
(1, 'Tennis Court 2', 5, 25, 8000, 200)
(2, 'Badminton Court', 0, 15.5, 4000, 50)
(3, 'Table Tennis', 0, 5, 320, 10)
(4, 'Massage Room 1', 9.9, 80, 4000, 3000)
(5, 'Massage Room 2', 9.9, 80, 4000, 3000)
(6, 'Squash Court', 3.5, 17.5, 5000, 80)
(7, 'Snooker Table', 0, 5, 450, 15)
(8, 'Pool Table', 0, 5, 400, 15)


In [53]:
# Q1

get_data(db_file, "SELECT name FROM FACILITIES WHERE membercost = 0")

['name']
('Badminton Court',)
('Table Tennis',)
('Snooker Table',)
('Pool Table',)


In [57]:
# Q2

get_data(db_file, "SELECT COUNT(DISTINCT name) as facilities FROM FACILITIES where membercost = 0")

['facilities']
(4,)


In [60]:
# Q3

get_data(db_file, "SELECT facid, name, membercost, monthlymaintenance FROM FACILITIES WHERE membercost < (monthlymaintenance * 0.2)")

['facid', 'name', 'membercost', 'monthlymaintenance']
(0, 'Tennis Court 1', 5, 200)
(1, 'Tennis Court 2', 5, 200)
(2, 'Badminton Court', 0, 50)
(3, 'Table Tennis', 0, 10)
(4, 'Massage Room 1', 9.9, 3000)
(5, 'Massage Room 2', 9.9, 3000)
(6, 'Squash Court', 3.5, 80)
(7, 'Snooker Table', 0, 15)
(8, 'Pool Table', 0, 15)


In [65]:
# Q4

get_data(db_file, "SELECT * FROM FACILITIES WHERE facid In (1,5)")

['facid', 'name', 'membercost', 'guestcost', 'initialoutlay', 'monthlymaintenance']
(1, 'Tennis Court 2', 5, 25, 8000, 200)
(5, 'Massage Room 2', 9.9, 80, 4000, 3000)


In [68]:
# Q5

get_data(db_file, "SELECT name, monthlymaintenance, CASE WHEN monthlymaintenance > 100 then 'expensive' ELSE 'cheap' END as label FROM FACILITIES")

['name', 'monthlymaintenance', 'label']
('Tennis Court 1', 200, 'expensive')
('Tennis Court 2', 200, 'expensive')
('Badminton Court', 50, 'cheap')
('Table Tennis', 10, 'cheap')
('Massage Room 1', 3000, 'expensive')
('Massage Room 2', 3000, 'expensive')
('Squash Court', 80, 'cheap')
('Snooker Table', 15, 'cheap')
('Pool Table', 15, 'cheap')


In [74]:
# Q6

get_data(db_file, "with cte as (SELECT *, ROW_NUMBER() OVER(ORDER BY joindate DESC) rn FROM members) SELECT firstname, surname FROM cte WHERE rn = 1 ")

['firstname', 'surname']
('Darren', 'Smith')


In [82]:
# Q7 

get_data(db_file, "SELECT facilities.name, firstname || ' ' || surname as memname  FROM members JOIN bookings ON members.memid = bookings.memid JOIN facilities ON bookings.facid = facilities.facid GROUP BY members.memid ORDER BY memname")

['name', 'memname']
('Badminton Court', 'Anna Mackenzie')
('Tennis Court 1', 'Anne Baker')
('Tennis Court 2', 'Burton Tracy')
('Tennis Court 1', 'Charles Owen')
('Table Tennis', 'Darren Smith')
('Tennis Court 1', 'David Farrell')
('Tennis Court 2', 'David Jones')
('Snooker Table', 'David Pinker')
('Badminton Court', 'Douglas Jones')
('Badminton Court', 'Erica Crumpet')
('Badminton Court', 'Florence Bader')
('Squash Court', 'GUEST GUEST')
('Tennis Court 1', 'Gerald Butters')
('Snooker Table', 'Henrietta Rumney')
('Badminton Court', 'Henry Worthington-Smyth')
('Snooker Table', 'Hyacinth Tupperware')
('Massage Room 1', 'Jack Smith')
('Massage Room 1', 'Janice Joplette')
('Table Tennis', 'Jemima Farrell')
('Snooker Table', 'Joan Coplin')
('Tennis Court 1', 'John Hunt')
('Massage Room 2', 'Matthew Genting')
('Badminton Court', 'Millicent Purview')
('Badminton Court', 'Nancy Dare')
('Tennis Court 2', 'Ponder Stibbons')
('Tennis Court 2', 'Ramnaresh Sarwin')
('Tennis Court 2', 'Tim Boothe')
(

In [101]:
# Q8

get_data(db_file, "SELECT name, firstname || ' ' || surname as memname, (CASE WHEN bookings.memid = 0 THEN guestcost*slots ELSE membercost*slots END) as cost FROM members JOIN bookings ON members.memid = bookings.memid JOIN facilities ON bookings.facid = facilities.facid where date(starttime) = '2012-09-14' AND cost > 30 GROUP BY bookid ORDER BY cost desc")

['name', 'memname', 'cost']
('Massage Room 2', 'GUEST GUEST', 320)
('Massage Room 1', 'GUEST GUEST', 160)
('Massage Room 1', 'GUEST GUEST', 160)
('Massage Room 1', 'GUEST GUEST', 160)
('Tennis Court 2', 'GUEST GUEST', 150)
('Tennis Court 1', 'GUEST GUEST', 75)
('Tennis Court 1', 'GUEST GUEST', 75)
('Tennis Court 2', 'GUEST GUEST', 75)
('Squash Court', 'GUEST GUEST', 70.0)
('Massage Room 1', 'Jemima Farrell', 39.6)
('Squash Court', 'GUEST GUEST', 35.0)
('Squash Court', 'GUEST GUEST', 35.0)


In [103]:
#Q9 

get_data(db_file, "SELECT name, memname, cost FROM (SELECT date(starttime) as date, name, firstname || ' ' || surname as memname, (CASE WHEN bookings.memid = 0 THEN guestcost*slots ELSE membercost*slots END) as cost FROM members JOIN bookings ON members.memid = bookings.memid JOIN facilities ON bookings.facid = facilities.facid GROUP BY bookid) cte WHERE date = '2012-09-14' AND cost > 30 ORDER BY  cost desc")

['name', 'memname', 'cost']
('Massage Room 2', 'GUEST GUEST', 320)
('Massage Room 1', 'GUEST GUEST', 160)
('Massage Room 1', 'GUEST GUEST', 160)
('Massage Room 1', 'GUEST GUEST', 160)
('Tennis Court 2', 'GUEST GUEST', 150)
('Tennis Court 1', 'GUEST GUEST', 75)
('Tennis Court 1', 'GUEST GUEST', 75)
('Tennis Court 2', 'GUEST GUEST', 75)
('Squash Court', 'GUEST GUEST', 70.0)
('Massage Room 1', 'Jemima Farrell', 39.6)
('Squash Court', 'GUEST GUEST', 35.0)
('Squash Court', 'GUEST GUEST', 35.0)


In [ ]:
#Q10 

get_data(db_file, "SELECT name, (CASE WHEN bookings.memid = 0 THEN guestcost*slots ELSE membercost*slots END) as tot_revenue FROM members JOIN bookings ON members.memid = bookings.memid JOIN facilities ON bookings.facid = facilities.facid WHERE tot

In [117]:
#Q10 

get_data(db_file, "SELECT name, tot_revenue FROM (SELECT name, SUM(CASE WHEN bookings.memid = 0 THEN guestcost*slots ELSE membercost*slots END) as tot_revenue FROM members JOIN bookings ON members.memid = bookings.memid JOIN facilities ON bookings.facid = facilities.facid GROUP BY 1) cte WHERE tot_revenue < 1000")

['name', 'tot_revenue']
('Pool Table', 270)
('Snooker Table', 240)
('Table Tennis', 180)


In [119]:
#Q11

get_data(db_file, "SELECT m1.memid, m1.surname, m1.firstname, m2.firstname || ' ' || m2.surname as recommendedby FROM members m1 JOIN members m2 ON m2.memid = m1.recommendedby ")

['memid', 'surname', 'firstname', 'recommendedby']
(4, 'Joplette', 'Janice', 'Darren Smith')
(5, 'Butters', 'Gerald', 'Darren Smith')
(7, 'Dare', 'Nancy', 'Janice Joplette')
(8, 'Boothe', 'Tim', 'Tim Rownam')
(9, 'Stibbons', 'Ponder', 'Burton Tracy')
(10, 'Owen', 'Charles', 'Darren Smith')
(11, 'Jones', 'David', 'Janice Joplette')
(12, 'Baker', 'Anne', 'Ponder Stibbons')
(14, 'Smith', 'Jack', 'Darren Smith')
(15, 'Bader', 'Florence', 'Ponder Stibbons')
(16, 'Baker', 'Timothy', 'Jemima Farrell')
(17, 'Pinker', 'David', 'Jemima Farrell')
(20, 'Genting', 'Matthew', 'Gerald Butters')
(21, 'Mackenzie', 'Anna', 'Darren Smith')
(22, 'Coplin', 'Joan', 'Timothy Baker')
(24, 'Sarwin', 'Ramnaresh', 'Florence Bader')
(26, 'Jones', 'Douglas', 'David Jones')
(27, 'Rumney', 'Henrietta', 'Matthew Genting')
(29, 'Worthington-Smyth', 'Henry', 'Tracy Smith')
(30, 'Purview', 'Millicent', 'Tracy Smith')
(35, 'Hunt', 'John', 'Millicent Purview')
(36, 'Crumpet', 'Erica', 'Tracy Smith')


In [137]:
# Q12 

get_data(db_file, "SELECT memid, SUM(slots) as usage, name FROM (SELECT name, bookings.memid as memid, slots FROM members JOIN bookings ON members.memid = bookings.memid JOIN facilities ON bookings.facid = facilities.facid WHERE bookings.memid IS NOT 0) cte GROUP BY 1,3 ORDER BY 2 DESC")

['memid', 'usage', 'name']
(1, 432, 'Badminton Court')
(3, 282, 'Pool Table')
(3, 176, 'Massage Room 1')
(5, 171, 'Tennis Court 1')
(8, 168, 'Tennis Court 2')
(3, 150, 'Table Tennis')
(4, 142, 'Snooker Table')
(10, 141, 'Tennis Court 2')
(12, 114, 'Tennis Court 2')
(12, 110, 'Squash Court')
(2, 102, 'Badminton Court')
(11, 99, 'Tennis Court 2')
(9, 96, 'Tennis Court 2')
(21, 96, 'Badminton Court')
(16, 95, 'Pool Table')
(2, 93, 'Tennis Court 1')
(6, 93, 'Tennis Court 1')
(2, 90, 'Snooker Table')
(8, 90, 'Snooker Table')
(15, 86, 'Table Tennis')
(11, 84, 'Tennis Court 1')
(21, 83, 'Pool Table')
(7, 81, 'Tennis Court 1')
(6, 78, 'Squash Court')
(8, 76, 'Massage Room 1')
(14, 75, 'Tennis Court 1')
(5, 72, 'Snooker Table')
(13, 68, 'Massage Room 1')
(5, 66, 'Massage Room 1')
(15, 66, 'Snooker Table')
(2, 64, 'Pool Table')
(5, 63, 'Badminton Court')
(6, 62, 'Massage Room 1')
(1, 58, 'Massage Room 1')
(1, 57, 'Tennis Court 2')
(4, 57, 'Tennis Court 1')
(1, 56, 'Table Tennis')
(2, 56, 'Table 

In [146]:
get_data(db_file, "SELECT month, SUM(slots) as usage, name FROM (SELECT name, slots, strftime('%m', starttime)  as month FROM members JOIN bookings ON members.memid = bookings.memid JOIN facilities ON bookings.facid = facilities.facid WHERE bookings.memid IS NOT 0) cte GROUP BY 1,3 ORDER By 2 DESC")

['month', 'usage', 'name']
('09', 507, 'Badminton Court')
('09', 443, 'Pool Table')
('09', 417, 'Tennis Court 1')
('08', 414, 'Badminton Court')
('09', 414, 'Tennis Court 2')
('09', 404, 'Snooker Table')
('09', 402, 'Massage Room 1')
('09', 400, 'Table Tennis')
('08', 345, 'Tennis Court 2')
('08', 339, 'Tennis Court 1')
('08', 316, 'Massage Room 1')
('08', 316, 'Snooker Table')
('08', 303, 'Pool Table')
('08', 296, 'Table Tennis')
('07', 201, 'Tennis Court 1')
('08', 184, 'Squash Court')
('09', 184, 'Squash Court')
('07', 166, 'Massage Room 1')
('07', 165, 'Badminton Court')
('07', 140, 'Snooker Table')
('07', 123, 'Tennis Court 2')
('07', 110, 'Pool Table')
('07', 98, 'Table Tennis')
('07', 50, 'Squash Court')
('09', 28, 'Massage Room 2')
('08', 18, 'Massage Room 2')
('07', 8, 'Massage Room 2')
